In [62]:
from itertools import permutations
from collections import Counter
import numpy as np

In [79]:
NUM_COLUMNS = 7
COLUMN_HEIGHT = 6
FOUR = 4
MAXLEVEL = 3
# Board can be initiatilized with `board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)`
# Notez Bien: Connect 4 "columns" are actually NumPy "rows"

In [64]:
def valid_moves(board):
    """Returns columns where a disc may be played"""
    valid = [n for n in range(NUM_COLUMNS) if board[n, COLUMN_HEIGHT - 1] == 0]
    #print("The valids are: ", valid)
    return valid


def play(board, column, player):
    """Updates `board` as `player` drops a disc in `column`"""
    (index,) = next((i for i, v in np.ndenumerate(board[column]) if v == 0))
    board[column, index] = player


def take_back(board, column):
    """Updates `board` removing top disc from `column`"""
    (index,) = [i for i, v in np.ndenumerate(board[column]) if v != 0][-1]
    board[column, index] = 0


def four_in_a_row(board, player):
    """Checks if `player` has a 4-piece line"""
    return (
        any(
            all(board[c, r] == player)
            for c in range(NUM_COLUMNS)
            for r in (list(range(n, n + FOUR)) for n in range(COLUMN_HEIGHT - FOUR + 1))
        )
        or any(
            all(board[c, r] == player)
            for r in range(COLUMN_HEIGHT)
            for c in (list(range(n, n + FOUR)) for n in range(NUM_COLUMNS - FOUR + 1))
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co, co + FOUR))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co + FOUR - 1, co - 1, -1))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
    )

In [65]:
def _mc(board, player):
    p = -player
    t=0
    while valid_moves(board):

        p = -p
        c = np.random.choice(valid_moves(board))
        play(board, c, p)

        if four_in_a_row(board, p):

            return p

    return 0


def montecarlo(board, player):
    montecarlo_samples = 50
    cnt = Counter(_mc(np.copy(board), player) for _ in range(montecarlo_samples))
    print("the count for 1 is: ", cnt[1], " while for -1 is: ",cnt[-1])
    print("the ret is:",(cnt[1] - cnt[-1]) / montecarlo_samples)
    return (cnt[1] - cnt[-1]) / montecarlo_samples


def eval_board(board):
    if four_in_a_row(board, 1):
        # Alice won
        return 1
    elif four_in_a_row(board, -1):
        # Bob won
        return -1
    else:
        # Not terminal
        return 0

In [69]:
def minmaxCF(board, p, layer):

    val = eval_board(board)

    valid = valid_moves(board)
    

    if val != 0 or not valid:
        return None, val
    evaluations = list()

    if(layer==MAXLEVEL):
            mnt = montecarlo(board, p)
            return None, mnt

    for col in valid:

        play(board, col, p)
        _, val = minmaxCF(-board, p, layer+1)
        evaluations.append((col, -val))
    if(p==1):
        #"# print("PLAYER 1 eval: ", evaluations)
        maxd = max(evaluations, key=lambda k: k[1])
        #"# print("PLAYER 1 max(val): ", maxd)
        return maxd
    else:
        #"# print("PLAYER -1 eval: ", evaluations)
        mind = min(evaluations, key=lambda k: k[1])
        #"# print("PLAYER -1 min(eval): ", mind)
        return mind

In [78]:
board = board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)
board2 = board
play(board, 1, -1)
play(board, 1, -1)
play(board, 4, 1)
play(board, 3, 1)
play(board, 1, -1)
print(board)
p = 1
layer = 0
i = 0
o =list()

while(i <= 5):
    output = minmaxCF(board, p, layer)

    o.append("loop nro "+str(i)+" minmax= "+ str(output))

    i = i+1
    layer=0


print(o)

[[ 0  0  0  0  0  0]
 [-1 -1 -1  0  0  0]
 [ 0  0  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]]
['loop nro 0 minmax= (5, 1)', 'loop nro 1 minmax= (None, 1)', 'loop nro 2 minmax= (None, 1)', 'loop nro 3 minmax= (None, 1)', 'loop nro 4 minmax= (None, 1)', 'loop nro 5 minmax= (None, 1)', 'loop nro 6 minmax= (None, 1)', 'loop nro 7 minmax= (None, 1)', 'loop nro 8 minmax= (None, 1)', 'loop nro 9 minmax= (None, 1)', 'loop nro 10 minmax= (None, 1)']
